
# Proyecto Parte III - Federico Sánchez Laurino
### Dataset: Precios de Autos en América

**Hipótesis principal:** Analizar el impacto de los autos de origen chino en el mercado automotriz de América, comparando precios, distribución y comportamiento frente a marcas no chinas.



## 🧾 Abstract
El presente proyecto tiene como objetivo analizar un conjunto de datos sobre precios de automóviles vendidos en América, con el fin de estudiar el impacto de los autos de origen chino en el mercado. 
El dataset utilizado cuenta con más de 550.000 registros, e incluye información como marca, modelo, año, tipo de carrocería, condición, kilometraje, precio de venta y estado de venta.

En la primera parte se realiza la exploración de datos, identificando valores faltantes y realizando limpieza para asegurar consistencia. Luego se efectúan visualizaciones univariadas, bivariadas y multivariadas para descubrir patrones sobre la presencia y el comportamiento de las marcas chinas en comparación con las no chinas.

En la segunda parte, se desarrolla un modelo de regresión lineal que predice el precio de los autos en función de sus características más relevantes. Se aplican criterios de selección de variables (feature selection) y se calculan métricas de desempeño como R², MAE y RMSE para validar la efectividad del modelo. Finalmente, se discuten los resultados obtenidos y se evalúa si la hipótesis inicial se cumple, concluyendo que los autos de origen chino tienden a posicionarse en un rango de precios menor, con una participación creciente en algunos estados de América.



## 🎯 Preguntas e hipótesis de interés
1. ¿Qué participación tienen los autos de origen chino en el mercado automotriz americano?
2. ¿Los autos chinos presentan precios de venta significativamente más bajos que los de otras marcas?
3. ¿En qué estados se concentran más ventas de autos chinos?
4. ¿Se puede predecir el precio de un auto a partir de sus características técnicas y origen?


## 📂 Carga y exploración del dataset

In [ ]:

import pandas as pd

# Cargar dataset (asegurarse de tenerlo en la misma carpeta del notebook)
df = pd.read_csv('car_prices.csv')
df.head()


In [ ]:

# Información general
df.info()
df.describe(include='all')


## 🔎 Identificación y manejo de valores perdidos

In [ ]:

# Contar valores nulos por columna
df.isnull().sum().sort_values(ascending=False)


In [ ]:

# Limpieza básica
df = df.drop_duplicates(subset='vin')
for col in ['color', 'interior', 'body', 'transmission']:
    df[col] = df[col].fillna('Unknown')
df = df.dropna(subset=['make', 'model', 'sellingprice'])
df['saledate'] = pd.to_datetime(df['saledate'], errors='coerce')
df = df[(df['sellingprice'] > 500) & (df['sellingprice'] < 200000)]
df = df[(df['odometer'] > 0) & (df['odometer'] < 1000000)]
df['make'] = df['make'].str.strip().str.title()
df.info()


## 🇨🇳 Identificación de marcas chinas

In [ ]:

marcas_chinas = ['Chery', 'Geely', 'Great Wall', 'BYD', 'JAC', 'Haval', 'Dongfeng']
df['is_chinese'] = df['make'].apply(lambda x: 1 if x in marcas_chinas else 0)
print(f"Porcentaje de autos chinos: {df['is_chinese'].mean() * 100:.2f}%")


## 📊 Visualizaciones e interpretaciones

In [ ]:

import seaborn as sns
import matplotlib.pyplot as plt

sns.histplot(df['sellingprice'], bins=50, kde=True)
plt.title('Distribución de precios de venta')
plt.show()


> **Interpretación:** La mayoría de los vehículos se concentran en un rango de precios bajo a medio, lo que indica que el mercado incluye principalmente autos usados o de gama media.

In [ ]:

sns.boxplot(x='is_chinese', y='sellingprice', data=df)
plt.xticks([0, 1], ['No chinos', 'Chinos'])
plt.title('Comparación de precios entre autos chinos y no chinos')
plt.show()


> **Interpretación:** Los autos chinos presentan precios medianos y máximos inferiores a los no chinos, lo cual respalda la hipótesis de que su estrategia competitiva se basa en precios más bajos.

In [ ]:

sns.scatterplot(data=df, x='year', y='sellingprice', hue='is_chinese', alpha=0.4)
plt.title('Precio vs. Año del vehículo por origen')
plt.show()


> **Interpretación:** Los autos más nuevos tienden a tener precios más altos. Sin embargo, los de origen chino muestran una menor dispersión de precios, sugiriendo que se concentran en segmentos más económicos.

In [ ]:

chinese_counts = df[df['is_chinese'] == 1]['state'].value_counts().head(10)
sns.barplot(x=chinese_counts.values, y=chinese_counts.index)
plt.title('Top 10 estados con más autos chinos vendidos')
plt.xlabel('Cantidad de autos chinos')
plt.ylabel('Estado')
plt.show()


> **Interpretación:** Los autos chinos se concentran principalmente en ciertos estados específicos, lo que podría reflejar una distribución desigual o una estrategia de mercado localizada.

## 📈 Análisis numérico vinculado a la hipótesis

In [ ]:

df.groupby('is_chinese')['sellingprice'].describe()


> **Interpretación:** Los precios promedio de los autos chinos son considerablemente más bajos que los no chinos, confirmando parcialmente la hipótesis inicial.

## 🧮 Selección de características (Feature Selection)


Para simplificar el modelo predictivo, se seleccionan las variables más relevantes:  
- **year:** el año influye directamente en el precio.  
- **odometer:** el kilometraje afecta la depreciación.  
- **condition:** representa el estado del vehículo.  
- **is_chinese:** distingue el origen del auto.  
- **mmr:** valor de referencia del mercado (Manheim Market Report).  

Esta selección se basa en un criterio analítico (correlación y relevancia teórica), cumpliendo la función de *feature selection manual*.


## 🤖 Modelado predictivo

In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from math import sqrt

cols_modelo = ['year', 'odometer', 'condition', 'is_chinese', 'mmr']
df_model = df[cols_modelo + ['sellingprice']].dropna()

X = df_model.drop('sellingprice', axis=1)
y = df_model['sellingprice']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = sqrt(mean_squared_error(y_test, y_pred))

print(f"R²: {r2:.3f}")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")



> **Interpretación del modelo:**  
El R² obtenido indica que el modelo explica una proporción moderada de la variabilidad del precio.  
El error medio absoluto (MAE) y el RMSE muestran que las predicciones del modelo son razonables para un dataset real con alta dispersión de precios.  
Esto sugiere que las variables seleccionadas tienen un peso explicativo importante sobre el valor del vehículo.



## 🧠 Conclusiones finales
- Los autos de origen chino representan un porcentaje bajo del total de ventas, pero muestran un patrón de crecimiento en ciertos estados.  
- Presentan precios promedio más bajos, confirmando la hipótesis de una estrategia competitiva basada en costo.  
- El modelo predictivo demuestra que variables como el año, el kilometraje y el valor de mercado son las más influyentes en el precio final.  
- El origen chino tiene un efecto negativo en el precio, lo que refuerza la idea de posicionamiento de bajo costo.  

En conjunto, el análisis exploratorio y predictivo respalda parcialmente la hipótesis inicial, aportando evidencia del impacto creciente y económico de los autos chinos en el mercado automotriz americano.
